In [ ]:
#One specific application of Text classification - Sentiment Analysis
'''
1. Unsupervised Sentiment classfications - word based
  Textblob
  AFINN
  Vader
2. Supervised Sentiment Classification
  n-grams:
  Bow
  TFIDF
  LR
'''
'''
Text classification:
1. Unsupervised - exploit word level information
2. Supervused - Train a model end - end
3. Transfer learning - We fine tune (partial tuning) already trained model to work on our inputs
4. Few shot - you give few examples as ask it to replicate
5. Zero shot - you dont give examples also. But model predicts
'''

In [1]:
!pip install textblob
!pip install textsearch
!pip install contractions
!pip install afinn
import nltk
nltk.download('all')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 728.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53430 sha256=a182272bcf7e78039a9e883d9ec5ccb6924ed75c8d8093bd0f38981b22370bdb
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [2]:
import pandas as pd
import numpy as np
import nltk
import textblob
import contractions

In [3]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("movie_reviews.csv.bz2")

In [5]:
import pandas as pd
movie_reviews = pd.read_csv("movie_reviews.csv.bz2")

In [6]:
movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
print(movie_reviews['review'][1])

A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.


In [ ]:
á

In [ ]:
é

In [9]:
#Beautiful soup
from bs4 import BeautifulSoup
import tqdm
import re
import unicodedata

In [10]:
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub("[\r|\n|\r\n]+",'\n',stripped_text)
  return stripped_text

In [11]:
strip_html_tags(movie_reviews['review'][1])

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

In [13]:
stop_words = nltk.corpus.stopwords.words('english')

In [ ]:
#format - UTF-8

In [14]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode('ascii','ignore').decode('utf-8','ignore')
  return text

In [15]:
remove_accented_characters("Résume")

'Resume'

In [59]:
import string
import re
import tqdm
def pre_processing(doc):
  norm_corpus = []
  for text in tqdm.tqdm(doc):
    text = text.lower()
    text = strip_html_tags(text)
    text = remove_accented_characters(text)
    text = contractions.fix(text)
    text = re.sub(r'[a-zA-Z0-9\s]',' ',text,re.I)
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    tokens = nltk.word_tokenize(text)
    text = text.strip
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_tokens)
    text = re.sub(' +',' ',text)
    norm_corpus.append(text)
  return norm_corpus

In [72]:
reviews = np.array(movie_reviews['review'])
sentiment = np.array(movie_reviews['sentiment'])

In [61]:
pre_processing([movie_reviews['review'][100]])

100%|██████████| 1/1 [00:00<00:00, 341.69it/s]


['short film inspired soontobe full length feature spatula madness hilarious piece contends similar cartoons yielding multiple writers short film stars edward spatula fired job joins fight evil spoons premise allows funny content near beginning barely present remainder feature films 15minute running time absorbed oddball comedy small musical number unfortunately much else lies plot set really time show surely follows plot better many highbudget hollywood films film worth watching least times take expect deep story']

In [62]:
normalized_reviews = np.array(pre_processing(reviews))

  2%|▏         | 806/50000 [00:01<00:59, 826.47it/s]<ipython-input-10-5fac35f36e46>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 50000/50000 [01:07<00:00, 741.63it/s]


In [66]:
normalized_reviews.shape

(50000,)

In [68]:
sample_reviews = [1000,2000,13010]

In [63]:
for review, sentiment in zip(normalized_reviews[sample_reviews],sentiment[sample_reviews]):
  print("Review: {}".format(review))
  print("Sentiment {}".format(sentiment))
  print("the predicted sentiment is: {}".format(textblob.TextBlob(review).sentiment.polarity))
  print('-'*100)

Review: movie awful even bothered write review garbage say one boring films ever seenand acting bad boy plays main character really annoys got expression face movie want slap basically 80 movie slow motion shots skateboarders weird music utter shtapparently got write least 10 lines text submit comment use lines saying lead character got one faces want slapmeh give upthis movie sucks
Sentiment negative
the predicted sentiment is: -0.3393939393939393
----------------------------------------------------------------------------------------------------
Review: randed space 1972 mst3k version good tv movie pilot never made series astronaut finds trapped earths evil twin planet identical size mass orbiting plane earth opposite side sun well worn sf chestnut idea 2000 years old invented ancient greeks version counter world run orwellian perfect society totally inexplicable reasons everyone speaks english drives late model american cars escaping prisonlike hospital disruptive earthian chased ar

In [64]:
from afinn import Afinn
afn = Afinn(emoticons = True)

In [74]:
sample_reviews

[1000, 2000, 13010]

In [73]:
for review, sentiment in zip(normalized_reviews[sample_reviews],sentiment[sample_reviews]):
  print("Review: {}".format(review))
  print("Sentiment {}".format(sentiment))
  print("the predicted sentiment is: {}".format(afn.score(review)))
  print('-'*100)

Review: movie awful even bothered write review garbage say one boring films ever seenand acting bad boy plays main character really annoys got expression face movie want slap basically 80 movie slow motion shots skateboarders weird music utter shtapparently got write least 10 lines text submit comment use lines saying lead character got one faces want slapmeh give upthis movie sucks
Sentiment negative
the predicted sentiment is: -17.0
----------------------------------------------------------------------------------------------------
Review: randed space 1972 mst3k version good tv movie pilot never made series astronaut finds trapped earths evil twin planet identical size mass orbiting plane earth opposite side sun well worn sf chestnut idea 2000 years old invented ancient greeks version counter world run orwellian perfect society totally inexplicable reasons everyone speaks english drives late model american cars escaping prisonlike hospital disruptive earthian chased around southern 

In [75]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [78]:
def sentiment_analysis_vader(review, threshold = 0.2, verbose = False):
  analyser = SentimentIntensityAnalyzer()
  scores = analyser.polarity_scores(review)
  agg_score = scores['compound']
  final_sentiment = 'positive' if agg_score > threshold else "negative"
  return scores, final_sentiment

In [79]:
sentiment_analysis_vader(normalized_reviews[13010])

({'neg': 0.063, 'neu': 0.476, 'pos': 0.461, 'compound': 0.9912}, 'positive')

In [ ]:
#lexon - Word - Each word carries weightage
#lack of domain specialization
# review 1: Speakers - The Product is damn loud! - Positive
# review 2: Movie - The movie is damn loud! - Negative

#Supervised Text classification / Sentiment Analysis